In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [99]:
user_pv = pd.DataFrame.from_csv('user_post_votes.csv', index_col=None)
user_pv.head(10)

,UserId,UserViews,UserReputation,PostId,PostType,VoteType,PostCreation,VoteCreation
0,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-15T00:00:00.000
1,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-15T00:00:00.000
2,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
3,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
4,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
5,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-22T00:00:00.000
6,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-28T00:00:00.000
7,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-12-13T00:00:00.000
8,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2012-01-30T00:00:00.000
9,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2012-04-20T00:00:00.000


In [78]:
user_pv[((user_pv.PostType!=1) & (user_pv.PostType!=2)) | (user_pv.PostId).isnull()]

,UserId,UserViews,UserReputation,PostId,PostType,VoteType,PostCreation,VoteCreation


In [83]:
user_pv[user_pv.UserId==3]

,UserId,UserViews,UserReputation,PostId,PostType,VoteType,PostCreation,VoteCreation
0,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-15T00:00:00.000
1,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-15T00:00:00.000
2,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
3,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
4,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-16T00:00:00.000
5,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-22T00:00:00.000
6,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-11-28T00:00:00.000
7,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2011-12-13T00:00:00.000
8,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2012-01-30T00:00:00.000
9,3,44,1317,1,1,2,2011-11-15T20:45:51.037,2012-04-20T00:00:00.000


In [100]:

def do_rename(se,new_name):
    se.name = new_name
    return se

def post_counts(df,p_typeid=1):
    diff_counts = lambda df_l : (df_l.drop_duplicates().groupby('UserId').cumcount() + 1).reindex(df_l.index).ffill()
    if p_typeid == 2:
        colname = "AnsContributed"
    if not (df.PostType==p_typeid).any():
        return pd.Series([0 for e in range(df.shape[0])],index=df.index)
    return diff_counts(df[df.PostType==p_typeid][['UserId','PostId']]).reindex(df.index).ffill().fillna(0)

predictors_tseries = ["QContributed","AnsContributed","VotesRcvQ","VotesRcvAns"]
get_cols = lambda df : [post_counts(df), post_counts(df,p_typeid=2), (df.PostType==1).cumsum(), (df.PostType==2).cumsum()]
        
ff1 = lambda df : pd.concat([do_rename(get_c,name) for name,get_c in zip(predictors_tseries,get_cols(df))],axis=1)
user_tseries = user_pv.sort_values(by=["PostCreation","VoteCreation"]).groupby(['UserId']).apply(ff1).reset_index(level=[0],drop=False)

In [103]:
user_tseries[(user_tseries.QContributed.isnull())| (user_tseries.AnsContributed.isnull())]

,UserId,QContributed,AnsContributed,VotesRcvQ,VotesRcvAns


## Aggregate predictors Reputation against (Q-Ans)Contributions and votes on contributions

In [44]:
users = user_pv[['UserId','UserViews','UserReputation','PostId','PostType']]

In [45]:
predictors = ["Reputation","Views","QContributed","AnsContributed","VotesRcvQ","VotesRcvAns"]

pr_values = lambda df: [[df.UserReputation.iloc[0]],[df.UserViews.iloc[0]], [df[df.PostType==1]['PostId'].unique().size],
                        [df[df.PostType==2]['PostId'].unique().size], [sum(df.PostType==1)], [sum(df.PostType==2)]]
ff2 = lambda df : pd.DataFrame(data=dict(zip(predictors,pr_values(df))))
users = users.groupby(['UserId']).apply(ff).reset_index(level=[1],drop=True)
users.head(10)

,AnsContributed,QContributed,Reputation,Views,VotesRcvAns,VotesRcvQ
UserId,,,,,,
3,11,6,1317,44,90,60
5,1,2,205,5,4,12
6,0,6,460,11,0,69
8,3,0,226,6,11,0
9,1,0,111,0,1,0
10,7,1,708,25,56,12
12,57,100,6139,375,289,625
15,2,0,153,6,5,0
18,0,2,380,0,0,55


In [7]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [8]:
users.describe()

,AnsContributed,QContributed,Reputation,Views,VotesRcvAns,VotesRcvQ
count,1959.000000,1959.000000,1959.000000,1959.000000,1959.000000,1959.000000
mean,3.438999,1.370087,199.009699,3.849923,11.979581,5.527310
std,13.486316,8.339413,669.354065,17.781874,52.105374,34.421235
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,66.000000,1.000000,1.000000,0.000000
75%,2.000000,1.000000,146.000000,2.000000,5.000000,3.000000
max,361.000000,266.000000,15418.000000,375.000000,1250.000000,1091.000000


In [9]:
users.Reputation.quantile([.75, .85, .95])

0.75    146.0
0.85    213.3
0.95    701.1
Name: Reputation, dtype: float64

In [11]:
mod_ols = smf.ols(formula='Reputation ~ Views + AnsContributed + QContributed + VotesRcvAns + VotesRcvQ', data= users[users.Reputation>=210])
res = mod_ols.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:             Reputation   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 1.316e+04
Date:                Thu, 07 Apr 2016   Prob (F-statistic):               0.00
Time:                        09:32:27   Log-Likelihood:                -1800.7
No. Observations:                 299   AIC:                             3613.
Df Residuals:                     293   BIC:                             3636.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         63.1466      6.807      9.

In [69]:
df1 = pd.DataFrame({'tag':['A','A','B','A','C'],'id': [1,1,1,1,1]})
do_rename(df1.id,"dodo")

0    1
1    1
2    1
3    1
4    1
Name: dodo, dtype: int64

In [41]:
unique_count = (df1.drop_duplicates().groupby('id').cumcount() + 1).reindex(df1.index).ffill()

In [98]:
a = pd.Series([np.nan,2,np.nan,3,np.nan,2,3,3])
a.fillna(method='ffill').fillna(0)

0    0
1    2
2    2
3    3
4    3
5    2
6    3
7    3
dtype: float64

In [23]:
df1.drop_duplicates()

,id,tag
0,1,A
2,1,B
3,1,C
4,2,A
6,2,B
